In [1]:
import copy
import random

from get_data import get_data
from blocks import init_blocks, calc_factors, get_block_stat, get_block_penalty
from students import init_students, calc_weights, get_students_penalty, update_students, get_p0_students, get_students_stat, get_students_for_module
from planner import get_try, list_results

In [4]:
blocks = get_data('data/blocks.json')
students_tmp = get_data('data/priorities.json')
specials = get_data('data/specials.json')

N_blocks = len(blocks)
N_students = len(students_tmp)
N_tries = 10

init_blocks(blocks)

students = init_students(students_tmp, blocks)

calc_factors(blocks)

calc_weights(students, N_blocks)

best_blocks = []

for block_nr in range(1, N_blocks+1):
    best_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
    block_penalty = get_block_penalty(best_block, block_nr, students)
    students_penalty = get_students_penalty(students, block_nr)
    penalty = block_penalty + students_penalty

    for i in range(N_tries):
        try_block = get_try(blocks[str(block_nr)], block_nr, students, specials[str(block_nr)])
        block_penalty_try = get_block_penalty(try_block, block_nr, students)
        students_penalty_try = get_students_penalty(students, block_nr)
        penalty_try = block_penalty_try + students_penalty_try

        # if block_penalty_next < block_penalty:
        #     try_block = try_block_next
        #     block_penalty = block_penalty_next
            
        if penalty_try < penalty:
            best_block = try_block
            block_penalty = block_penalty_try
            students_penalty = students_penalty_try
            penalty = penalty_try
            
    update_students(best_block, block_nr, students)

    best_blocks.append(best_block)


In [14]:
block_nr = 5
details = True

list_results(best_blocks, block_nr, students, details)

block 5: p1: 36.5%, p2: 43.3%, p3: 14.6%, p0: 5.6%
Oberflächen und Farben: 10/20, p1: 30.0% (60.0%), p2: 50.0% (300.0%), p3: 20.0% (330.0%), p0: 0.0%
Fourier-Reihen: 3/20, p1: 66.7% (200.0%), p2: 0.0% (533.3%), p3: 33.3% (666.7%), p0: 0.0%
Datenexploration: 19/20, p1: 47.4% (57.9%), p2: 36.8% (100.0%), p3: 10.5% (115.8%), p0: 5.3%
Astrophysik, Planetologie und Raumfahrt: 20/20, p1: 50.0% (155.0%), p2: 40.0% (220.0%), p3: 10.0% (135.0%), p0: 0.0%
Molekulare Schalter – eine on/off-Beziehung: 12/12, p1: 8.3% (50.0%), p2: 75.0% (325.0%), p3: 16.7% (175.0%), p0: 0.0%
Let's Arduino: 11/12, p1: 27.3% (36.4%), p2: 54.5% (190.9%), p3: 0.0% (209.1%), p0: 18.2%
Die Hoffnung ist erneuerbar: 14/20, p1: 21.4% (21.4%), p2: 35.7% (164.3%), p3: 28.6% (214.3%), p0: 14.3%
Swiss Young Physicists' Tournament: 3/20, p1: 66.7% (333.3%), p2: 0.0% (133.3%), p3: 0.0% (500.0%), p0: 33.3%
Von Big Data und Künstlicher Intelligenz zur Umweltinformatik: 11/16, p1: 27.3% (72.7%), p2: 45.5% (272.7%), p3: 27.3% (218.2%

In [13]:
for module in best_blocks[4]:
    print(module, get_students_for_module(best_blocks[4], module, students))

Oberflächen und Farben ['3b', '3c', '3d', '3e', '3f']
Fourier-Reihen ['3a', '3c', '3e']
Datenexploration ['3a', '3c', '3d', '3f', '3g', '3h']
Astrophysik, Planetologie und Raumfahrt ['3a', '3d', '3e', '3f', '3g', '3h']
Molekulare Schalter – eine on/off-Beziehung ['3a', '3b', '3d', '3e', '3h']
Let's Arduino ['3a', '3b', '3c', '3d', '3f', '3g', '3h']
Die Hoffnung ist erneuerbar ['3b', '3c', '3d', '3e', '3f', '3g', '3h']
Swiss Young Physicists' Tournament ['3a', '3b', '3f']
Von Big Data und Künstlicher Intelligenz zur Umweltinformatik ['3a', '3c', '3d', '3e', '3g', '3h']
Woher kommst du wirklich? Frag deine DNA! ['3a', '3b', '3c', '3d', '3e', '3f', '3g']
Astronomie ['3a', '3b', '3c', '3d', '3e', '3f', '3g']
Entsorgung, Recycling und Abwasserreinigung in Zürich ['3a', '3b', '3c', '3d', '3e', '3g', '3h']
Praktische Molekulargenetik: Wolbachia – auf der Suche nach Bakterien-DNA in selbstgefangenen Insekt ['3a', '3b', '3c', '3d', '3e', '3h']
Energiespeicherung – Vom Akku bis zum Pump-Speicher